In [0]:
from delta.tables import *

stage_table_name = "incremental_load.default.orders_stage"
target_table_name = "incremental_load.default.orders_target"

In [0]:
# note: for the first time load, the target table will be empty so we load all the data from Staging Table to the Target Table

# read the data from the staging table
stage_df = spark.read.table(stage_table_name)

In [0]:
# create equivalent target table schema if target table does not exist

if not spark._jsparkSession.catalog().tableExists(target_table_name):
    stage_df.write.format("delta").mode("overwrite").saveAsTable(target_table_name)

else:
    # perform delta tanle merge query for upsert based on tracking_num column
    target_table = DeltaTable.forName(spark, target_table_name)

    # define the merge condition based on the tracking_num  column
    merge_condition = "stage.tracking_num = target.tracking_num"

    # execute the merge operation
    target_table.alias("target")\
         .merge(stage_df.alias("stage"), merge_condition)\
         .whenMatchedDelete() \
         .execute()

    stage_df.write.format("delta").mode("append").saveAsTable(target_table_name)
